In [1]:
import time
import torch
from cudaext.ops.trilinear_interpolate.trilinear_interpolate_utils import *
N = 65536
F = 256
rand = torch.rand(N, 8, F, device='cuda')
feats = rand.clone().requires_grad_()
feats2 = rand.clone().requires_grad_()
points = torch.rand(N, 3, device='cuda')*2-1
points1 = points.clone().requires_grad_()
points2 = points.clone().requires_grad_()

t = time.time()
out_cuda = Trilinear_interpolation_cuda.apply(feats2, points2)
torch.cuda.synchronize()
print('   cuda fw time', time.time()-t, 's')

t = time.time()
out_py = trilinear_interpolation_cpu(feats, points1)
torch.cuda.synchronize()
print('pytorch fw time', time.time()-t, 's')

print('fw all close', torch.allclose(out_py, out_cuda))

t = time.time()
loss2 = out_cuda.sum()
loss2.backward()
torch.cuda.synchronize()
print('   cuda bw time', time.time()-t, 's')

t = time.time()
loss = out_py.sum()
loss.backward()
torch.cuda.synchronize()
print('pytorch bw time', time.time()-t, 's')
print(points1.grad)
print(points2.grad)
print('bw all close', torch.allclose(feats.grad, feats2.grad))
print('bw all close', torch.allclose(points1.grad, points2.grad, atol = 1e-4))

   cuda fw time 0.007082223892211914 s
pytorch fw time 0.02019643783569336 s
fw all close True
   cuda bw time 0.04685664176940918 s
pytorch bw time 0.15795564651489258 s
tensor([[ 2.0943, -0.5042, -0.0591],
        [ 8.8969,  0.4554,  3.1382],
        [-4.4725, -3.8302, -1.1909],
        ...,
        [-0.3442,  0.5533,  1.2127],
        [-1.2295, -0.1724,  1.5378],
        [ 0.4441, -0.9800, -1.7823]], device='cuda:0')
tensor([[ 2.0943, -0.5042, -0.0591],
        [ 8.8969,  0.4554,  3.1382],
        [-4.4724, -3.8302, -1.1909],
        ...,
        [-0.3442,  0.5533,  1.2127],
        [-1.2295, -0.1724,  1.5378],
        [ 0.4441, -0.9800, -1.7823]], device='cuda:0')
bw all close True
bw all close True
